In [40]:
# estado_ini = ( (personas), (ascensores) )
estado_ini = (
    (1,4,3), (1, 2)
)

# estado_goal = (personas)
estado_goal = (3,2,2)

# Acciones
# Llamar ascensor (mueve un ascensor vacio a una planta)
# Subir persona (sube una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar persona (baja una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)

In [41]:
# importamos las cosas que vamos a usar de aima
from search import *

class Ascensores(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal):
        '''Inicializacion de nuestro problema.'''
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        personas = list(estado[0])
        ascensores = list(estado[1])
        accs = list()
        for i, persona in enumerate(personas):
            for j, ascensor in enumerate(ascensores):
                if persona != ascensor:
                    accs.append(("Llamar", j, i))            # Persona i llama a asccensor j
                if persona == ascensor and int(persona) > 0:
                    accs.append(("Bajar", j, i))             # Bajar ascensor j con persona i
                if persona == ascensor and int(persona) < 12:
                    accs.append(("Subir", j, i))             # Subir ascensor j con persona i
        return accs
    
    def result(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        a = accion[1]
        p = accion[2]
        
        if accion[0] == "Llamar":
            ascensores[a] = int(personas[p])
        elif accion[0] == "Bajar":
            ascensores[a] -= 1
            personas[p] -= 1
        elif accion[0] == "Subir":
            ascensores[a] += 1
            personas[p] += 1
            
        return (tuple(personas), tuple(ascensores))
    
    def goal_test(self, estado):
        return estado[0] == self.goal


In [42]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((1, 4, 3), (1, 2))

In [43]:
p.actions(p.initial)

[('Bajar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 0),
 ('Llamar', 0, 1),
 ('Llamar', 1, 1),
 ('Llamar', 0, 2),
 ('Llamar', 1, 2)]

In [44]:
p.result(p.initial, ("Subir", 0, 0))

((2, 4, 3), (2, 2))

In [45]:
p.goal

(3, 2, 2)

In [46]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [47]:
breadth_first_graph_search(p).solution()

[('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 0, 1),
 ('Bajar', 1, 2)]